In [1]:
import pandas as pd
import os
import re
import warnings
import openpyxl
from openpyxl.styles import Font, Alignment, Side, Border, PatternFill, GradientFill, PatternFill
from openpyxl.formatting.rule import DataBarRule
from openpyxl import load_workbook
from openpyxl import Workbook
from decimal import Decimal#保证比例呈现两位小数
from math import pow 
import tkinter as tk
from tkinter import filedialog
import keyboard
import sys
import numpy as np
warnings.filterwarnings('ignore')

# 读取数据并输出excel

In [2]:
def safe_read_excel(data_name,title_columns):
    for i in range(3):
        data=pd.read_excel(data_name,header=i,engine='openpyxl')
        data.columns=[isinstance(i,str) and re.sub('\s','',i) or '' for i in data.columns]
        tempg=[]
        for j in title_columns:
            for i in range(len(data.columns)):
                if j in data.columns[i]:
                    tempg.append(i)
                    break
        if len(tempg)==len(title_columns):
            return data.iloc[:,tempg]
    print(f'未在“{data_name}”的前三行中检索到列名{str(title_columns)}')
    return 

In [3]:
try:
    del root
except:
    pass
try:
    root = tk.Tk()
    root.withdraw()
    data_w_name = filedialog.askopenfilename(title=u'请选择往届的excel文件')
    data_d_name1 = filedialog.askopenfilename(title=u'请选择当届本专学生信息excel文件')
    data_d_name2 = filedialog.askopenfilename(title=u'请选择当届的研究生学生信息excel文件')
    data_z_name = filedialog.askopenfilename(title=u'请选择重点群体名单的excel文件')
    data_d_name = filedialog.askopenfilename(title=u'请选择上周导出的excel文件，没有则点击取消')
    path = filedialog.asksaveasfile(title=u'请选择输出的excel文件将要保存的位置',defaultextension='.xlsx')
    if data_d_name!='':
        data_a = safe_read_excel(data_d_name,['学院','专业','学历','就业率'])
    print('请等待......')
    path = path.name
    root.destroy()
    data_z = pd.read_excel(data_z_name)
    data_w = safe_read_excel(data_w_name,['学院','专业','学历','就业率'])
    data_d = pd.concat([pd.read_excel(data_d_name1),pd.read_excel(data_d_name2)],axis=0)
except PermissionError:
    # 处理 PermissionError 异常的代码
    print("请关闭excel后再尝试")
    raise
    keyboard.wait('enter')
    sys.exit()
except:
    print('未选择正确的文件!请按enter键退出后重试')
    raise
    keyboard.wait('enter')
    sys.exit()

请等待......


In [4]:
#对当届数据进行处理
data = data_d[['所在院系','专业','学历','毕业去向代码','身份证号']]
data.columns=['所在院系','专业','学历','毕业去向','身份证号']
data['学历']=data['学历'].apply(lambda x:'本'in x and '本科生' or('专'in x and '专科生' or '研究生'))
# 个性化处理
data['学历']=data[['学历','所在院系']].apply(lambda x:'MBA' in x['所在院系'] and 'MBA' or x['学历'],axis=1)
data['所在院系']=data['所在院系'].apply(lambda x:'MBA' in x and '经济管理学院' or x)

# 把后面的改成前面的
temp=[[['专业','材料与化工（纺织工程）'],['机电工程学院','材料与化工','研究生']],
      [['专业','市政工程（含给排水）'],['土木学院','市政工程(含给排水等)','研究生']],
      [['专业','法学'],['法学院','法学','研究生']],
      [['所在院系','计算机与通信学院'],['软件学院','软件技术','专科生']],
      [['所在院系','计算机与通信学院'],['软件学院','动漫制作技术','专科生']],
     ]
for i in temp:
    data[i[0][0]]=data.apply(lambda x:x['所在院系']==i[1][0] and x['专业']==i[1][1] and x['学历']==i[1][2] and i[0][1] or x[i[0][0]],axis=1)
    
#连接重点群体表
data=pd.merge(data,data_z[['身份证号','姓名']],on='身份证号',how='left')
jy_group=[75,46,502,76,11,515,272,512,803,50,271,10,511,514,12,27,801,519,503,802,85,501,51,80,513]
data['毕业去向落实']=data['毕业去向'].apply(lambda x:1 if x in jy_group else 0)
data['毕业去向']=1
data['是否重点']=data['姓名'].apply(lambda x:0 if isinstance(x,float) else 1)
data.drop(columns=['身份证号','姓名'],inplace=True)

In [5]:
data1 =data.groupby(['所在院系','专业','学历'])[['毕业去向','毕业去向落实']].sum().reset_index()
# 分学历院系总计
data2 =data.groupby(['所在院系','学历'])[['毕业去向','毕业去向落实']].sum().reset_index()
# 分院系重点群体总计
data3 =data[data['是否重点']==1].groupby(['所在院系'])[['毕业去向','毕业去向落实']].sum().reset_index()
# 分院系总计
data4 =data.groupby(['所在院系'])[['毕业去向','毕业去向落实']].sum().reset_index()
data2['专业']=data2['学历']+'总计'
data2['学历']=data2['专业']
data3['学历']='重点群体毕业生总计'
data3['专业']='重点群体毕业生总计'
data4['学历']='全院总计'
data4['专业']='全院总计'

In [6]:
temp=pd.concat([data1,data2,data3,data4],axis=0)

xl_order={'本科生':1,'专科生':2,'研究生':3,'MBA':3.5,'重点群体毕业生总计':4,'MBA总计':4.5,'专科生总计':6,'本科生总计':5,'研究生总计':7,'全院总计':8}
temp['xl']=temp['学历'].apply(lambda x:xl_order[x])
yx_order={'体育教学研究部':[16,'体育部'], '土木工程学院':[5,'土木学院'], '外国语学院':[12,'外语学院'], '文学院':[15,'文学院'],
          '新能源学院':[17,'新能源学院'], '机电工程学院':[2,'机电学院'],'材料科学与工程学院':[1,'材料学院'],
          '法学院':[14,'法学院'], '理学院':[11,'理学院'], '生命科学与工程学院':[10,'生命学院'], 
          '电气工程与信息工程学院':[6,'电信学院'], '石油化工学院':[3,'石化学院'],'经济管理学院':[8,'经管学院'],
          '能源与动力工程学院':[4,'能动学院'], '计算机与通信学院':[7,'计通学院'], '设计艺术学院':[9,'设计学院'], 
          '马克思主义学院':[13,'马院']}
yx_order_t={yx_order[i][1]:yx_order[i] for i in yx_order}
yx_order={**yx_order_t,**yx_order}

# 这里做了容错，匹配不上的学院也不会报错了
temp['yx']=temp['所在院系'].apply(lambda x:yx_order.get(x,[100,x])[0])
temp['所在院系']=temp['所在院系'].apply(lambda x:yx_order.get(x,[100,x])[1])
temp=temp.sort_values(by=['yx','xl']).drop(columns=['yx','xl'])
#去掉不需要的‘全院总计’
mask=(temp['毕业去向']!=temp['毕业去向'].shift(1))|(temp['学历']!='全院总计')
temp=temp[mask]

# 大总计
data5 = data2.copy()
data5['学历']=data5['学历'].apply(lambda x:x=='MBA总计' and '研究生总计' or x)
data5['专业']=data5['学历']
data5=data5.groupby(['学历','专业'])[['毕业去向','毕业去向落实']].sum().reset_index()
data5=pd.concat([data5,data3.groupby(['学历','专业'])[['毕业去向','毕业去向落实']].sum().reset_index(),data4.groupby(['学历','专业'])[['毕业去向','毕业去向落实']].sum().reset_index()],axis=0)
data5['xl']=data5['学历'].apply(lambda x:xl_order[x])
zz_group={'MBA总计':'MBA总计','重点群体毕业生总计':'重点群体毕业生总计','专科生总计':'全校专科生','本科生总计':'全校本科生','研究生总计':'全校研究生','全院总计':'全校总计'}
data5['所在院系']=data5['学历'].apply(lambda x:zz_group[x])
data5['学历']=data5['所在院系']
data5['专业']=data5['所在院系']
data5=data5.sort_values(by=['xl']).drop(columns=['xl'])
temp=pd.concat([temp,data5],axis=0)
temp['就业率']=round(temp['毕业去向落实']/temp['毕业去向'],4)

#算上周同期
if data_d_name!='':
    data6=data_a.copy()
    data6.columns=['所在院系','专业','学历','就业率']
    for i in range(data6.shape[0]):
        if isinstance(data6.iloc[i,0],float):
            data6.iloc[i,0]=data6.iloc[i-1,0]
        else:
            data6.iloc[i,0]=re.sub('\s','',data6.iloc[i,0])
    data6['专业']=data6.apply(lambda x:isinstance(x['专业'],float) and x['所在院系'] or x['专业'],axis=1)
    data6['学历']=data6.apply(lambda x:isinstance(x['学历'],float) and x['专业'] or x['学历'],axis=1)
    temp=pd.merge(temp,data6,on=['所在院系','专业','学历'],how='left')

#算去年同期
data6=data_w.copy()
data6=data6.applymap(lambda x:'重点群体毕业生总计' if isinstance(x,str) and '重点群体' in x else x)
data6.columns=['所在院系','专业','学历','就业率']
data6['学历']=data6['学历'].apply(lambda x:'本科生' if isinstance(x,str) and '本科' in x else x)
for i in range(data6.shape[0]):
    if isinstance(data6.iloc[i,0],float):
        data6.iloc[i,0]=data6.iloc[i-1,0]
    else:
        data6.iloc[i,0]=re.sub('\s','',data6.iloc[i,0])
data6['专业']=data6.apply(lambda x:isinstance(x['专业'],float) and x['所在院系'] or x['专业'],axis=1)
data6['学历']=data6.apply(lambda x:isinstance(x['学历'],float) and x['专业'] or x['学历'],axis=1)
temp=pd.merge(temp,data6,on=['所在院系','专业','学历'],how='left')

#差值需要相减，要保证都是float
def safe_cut(x,y):
    try:
        x=(isinstance(x,str) and '%'  in str(x) and float(re.sub('%','',str(x)))/100) or x
        y=(isinstance(y,str) and '%'  in str(y) and float(re.sub('%','',str(y)))/100) or y
        return x-y
    except:
        return '/'
    
def safe_temp(x):
    if not pd.notna(x):return '/'#处理np.nan
    #处理带%字符串或非nan浮点数,其他都是/
    y=lambda x:(isinstance(x,str) and '%' in x and x) or ((isinstance(x,float) and str(Decimal(x*100).quantize(Decimal("0.00")))+'%')) or '/'
    try:
        return y(x)
    except:
        pass#理论上来说不可能报错了
if data_d_name=='':
    temp.columns=['学院','专 业 方 向','学历','学生总数','就业人数','就业率','去年同期\n就业率']
    temp['就业率差值\n（同期）']=temp.apply(lambda x:safe_cut(x['就业率'],x['去年同期\n就业率']),axis=1)
else:
    temp.columns=['学院','专 业 方 向','学历','学生总数','就业人数','就业率','就业率\n（上周）','去年同期\n就业率']
    temp['就业率差值\n（上周）']=temp.apply(lambda x:safe_cut(x['就业率'],x['就业率\n（上周）']),axis=1)
    temp['就业率差值\n（同期）']=temp.apply(lambda x:safe_cut(x['就业率'],x['去年同期\n就业率']),axis=1)

for i in temp.columns[5:]:
    temp[i]=temp[i].apply(lambda x:safe_temp(x))
    temp[i]=temp[i].apply(lambda x:x=='-0.00%' and '0.00%' or x)

In [7]:
data_all = temp.copy()
data_all.to_excel(path,index=False,header=False)

# 修改excel格式

In [8]:
h,w=data_all.shape

In [9]:
def nc(c,i):
    return chr(ord(c)+i)
def pf(n):
    return pow(n,2)
font1 = Font(bold=False,size=12,name='宋体')
font2 = Font(bold=True,size=12,name='黑体')
font3 = Font(bold=True,size=12,name='Tahoma')
font4 = Font(bold=False,size=12,name='Tahoma')
side=Side(style='thin', color='000000')
border1 = Border(left=side,right=side,top=side,bottom=side)
alignment1 = Alignment(horizontal='center', vertical='center')

In [10]:
workbook = load_workbook(filename=path)
worksheet = workbook.active

for i in range(h+3):
    worksheet.row_dimensions[i+1].height=15
worksheet.insert_rows(1)
temp1=data_all.columns
for i in range(len(temp1)):
    temp=nc('A',i)
    cell=worksheet[f'{temp}1']
    worksheet[f'{temp}1']=temp1[i]
    cell.font=font2
    cell.alignment=Alignment(horizontal='center', vertical='center', wrapText=True)
    cell.border=border1
    
worksheet.insert_rows(1)
temp={'A':pf(7.25)/6.62,'B':pf(28.13)/27.51,'C':pf(9.88)/9.26,'D':pf(9.88)/9.26,'E':pf(11)/10.38,'F':pf(11)/10.38,'G':pf(12.25)/11.63,'H':pf(13.5)/12.88}
if data_d_name!='':
    temp['I']=pf(13.5)/12.88
    temp['J']=pf(13.5)/12.88
for i in temp:
    worksheet.column_dimensions[i].width = temp[i]#宽度
    
# 批量改数据格式
for i in range(1,h+2):
    for j in range(w):
        temp=nc('A',j)
        cell=worksheet[f'{temp}{i+2}']
        cell.border = border1
        cell.alignment = alignment1
        cell.font=font1
        if j>2 and j<=4:cell.font=font4
        if j>4:cell.font=font3
        if j==0 or cell.value in zz_group or cell.value in zz_group.values():cell.font=font2
            
# 合并相同的单元格
temp=0
for i in range(2,h+3):
    if worksheet[f'B{i}'].value==worksheet[f'C{i}'].value:
        if worksheet[f'B{i}'].value!=worksheet[f'A{i}'].value:
            worksheet.merge_cells(f'B{i}:C{i}')
        else:
            worksheet.merge_cells(f'A{i}:C{i}')
    if worksheet[f'A{i}'].value==worksheet[f'A{i+1}'].value:
        temp+=1
    else:
        worksheet.merge_cells(f'A{i-temp}:A{i}')
        cell=worksheet[f'A{i-temp}']
        yx_enter={'体育部':'体育部', '土木学院':'土木\n学院', '外语学院':'外语\n学院', '文学院':'文学院',
          '新能源学院':'新能源\n学院', '机电学院':'机电\n学院','材料学院':'材料\n学院',
          '法学院':'法\n学院', '理学院':'理\n学院', '生命学院':'生命\n学院', 
          '电信学院':'电信\n学院', '石化学院':'石化\n学院','经管学院':'经管\n学院',
          '能动学院':'能动\n学院', '计通学院':'计通\n学院', '设计学院':'设计\n学院', '马院':'马院'}
        try:
            cell.value=yx_enter[cell.value]
        except:
            pass
        cell.alignment = Alignment(horizontal='center', vertical='center', wrapText=True)
        temp=0
    
temp=nc('A',w-1)
worksheet.merge_cells(f'A{h+3}:{temp}{h+3}')
worksheet[f'A{h+3}']='                                                           数据来源：全国高校毕业生就业管理系统'
#标题处理
worksheet.row_dimensions[1].height=57
worksheet.row_dimensions[2].height=65
temp=nc('A',w-1)
worksheet.merge_cells(f'A1:{temp}1')
cell=worksheet[f'A1']
cell.value='兰州理工大学20XX届毕业生就业情况统计表\n（时间截至X月XX日12:00）'
cell.alignment = Alignment(horizontal='center', vertical='center', wrapText=True)
cell.font=Font(bold=True,size=16,name='宋体')
            
workbook.save(filename=path)

print('流程结束，请按enter键退出')
keyboard.wait('enter')

流程结束，请按enter键退出
